In [ ]:
# Importamos librerías externas: 

# Para arrancar el proyecto, necesitamos nuestras herramientas principales, que son las librerías que amplían las capacidades de Python.
# Las librerías son herramientas creadas por la comunidad que amplían Python y se instalan con pip.

# Conectar y manejar Bases de Datos MySQL. (Necesario para Fases 2 y 3)
import mysql.connector

# El motor matemático: maneja números y cálculos a gran velocidad. (Útil, aunque no esencial para este ejercicio porque no haremos cálculos complejos)
import numpy as np

# El Excel de Python: organiza y analiza datos en forma de tablas (DataFrames). (Esencial para guardar las películas)
import pandas as pd

# para poder visualizar todas las columnas de los DataFrames (Muy útil cuando trabajas con DataFrames de Pandas que tienen muchas columnas)
pd.set_option('display.max_columns', None) 

# El cartero de Internet: pide y envía información a páginas web y servidores. (Esencial para descargar la API)
import requests

# Manual de "Qué hacer si falla": identifica problemas al conectar a MySQL. (Buena práctica para manejar errores)
from mysql.connector import Error

In [ ]:
# FASE 1: EXTRACCIÓN DE DATOS DE PELICULAS DESDE LA API

# Guardamos la variable 'url' la dirección web donde se encuentran los datos (la API).
url = "https://beta.adalab.es/resources/apis/pelis/pelis.json"

print(f"Intentando conectar a la API en: {url}")

# Utilizamos un bloque try/except para manejar posibles fallos de conexión a Internet.
try:
    # Hacemos una petición GET a la URL usando la librería 'requests' para descargar los datos.
    datos = requests.get(url)

    # Consultamos el código de respuesta del servidor (200 = OK).
    if datos.status_code == 200:
        print("Conexión exitosa. Código de estado:", datos.status_code)

        # Vemos el contenido de la respuesta codificada en bytes.
        datos.content
        
        # Con JSON, primero, se descodifica la secuencia de bytes y luego se estructuran los datos ( lista o diccionario)
        datos_pelis = datos.json()

        # Visualizamos la estructura de los datos descargados.
        print(f"Estructura de los datos descargados en formato json:{datos_pelis}")

    else:
        # Si el código de estado no es 200, mostramos un mensaje de error.
        print(f"Error en la conexión. Código de estado: {datos.status_code}. No se pudieron extraer los datos.")

# Cuando se produzcan estas situaciones: 1) No tener conexión a Internet (el error más común). 2) Escribir una URL mal formada (por ejemplo, faltan caracteres). 3) Que el servidor de la API esté caído o no responda a tiempo.
# Se encuentra dentro de la documentación oficial de la librería requests de Python
except requests.exceptions.RequestException as e:
    print(f"Ocurrió un error al intentar conectar a la API: {e}")

In [ ]:
# Aunque sabemos que el tipo de dato es una lista porque lo hemos visto al imprimir datos_pelis, generalmente no es recomendable eliminar la verificación del tipo de dato.
print(f"El tipo de dato es: {type(datos_pelis)}")

In [ ]:
# Usamos pandas para crear el DataFrame (el "Excel de Python") a partir de la lista de películas.
df_peliculas = pd.DataFrame(datos_pelis)

# Mostramos las filas del Dataframe para verificar que se han cargado correctamente.
print(f"Filas del Dataframe de películas:{df_peliculas}") 

# Muestra el número de películas extraídas (deberían ser 100).
print(f"El número de películas extraídas es: {len(df_peliculas)}")

In [ ]:
# Este código me modifica la columa subtitulos, y es una lista, pero con astype: list --> str
df_peliculas['subtitulos'] = df_peliculas['subtitulos'].astype(str)

df_peliculas['subtitulos']

In [ ]:
# Verificamos la versión de la librería mysql-connector-python instalada.
print(f"Versión: {mysql.connector.__version__}")

In [ ]:
# FASE 2: CREACIÓN DE LA BASE DE DATOS (VERSIÓN PYTHON)
# A. Conexión a MySQL
try:
    # Intentamos conectarnos al servidor MySQL
    cnx = mysql.connector.connect(
        host='127.0.0.1',    # Dirección del servidor MySQL (localhost)
        user='root',         # Usuario con el que nos conectamos
        password='AdalabAlumna',  # Contraseña del usuario 
        use_pure=True, # Importante para Python 3.12 ( necesario para no romperse el Kernel)
    )
    print('Conexión exitosa')  # Si no hay errores, mostramos que la conexión fue correcta
except Error as e:  # Captura cualquier error relacionado con MySQL
    print('Error al conectar:', e)  # Muestra el mensaje de error para entender qué falló

In [ ]:
# B. Creación de la base de datos

try:
    # Creamos un cursor (intermediario entre Python y MySQL)
    mycursor = cnx.cursor()

    # Consulta SQL para crear la base de datos si no existe
    query_create_database = "CREATE DATABASE IF NOT EXISTS peliculas_db"

    # Ejecutamos la consulta SQL
    mycursor.execute(query_create_database)
    print("Base de datos creada correctamente (o ya existía).")

except mysql.connector.Error as e:
    print("Ocurrió un error al crear la base de datos:", e)

In [ ]:
# C. Creación de las tablas

try:
    # Seleccionamos la base de datos donde vamos a trabajar
    mycursor.execute("USE peliculas_db")

    # Comando SQL para crear la tabla 'peliculas' si no existe
    query_creacion_tabla = """
    CREATE TABLE IF NOT EXISTS peliculas (
        id INT AUTO_INCREMENT PRIMARY KEY,   # Identificador único
        titulo VARCHAR(255) NOT NULL,        # Título obligatorio
        año YEAR,                            # Año de estreno
        duracion INT,                        # Duración en minutos
        genero VARCHAR(100),                 # Género de la película
        adultos TINYINT(1),                  # 0 = No adultos, 1 = Adultos
        subtitulos VARCHAR(255)              # Idioma de subtítulos
    );
    """

    # Ejecutamos la consulta SQL para crear la tabla
    mycursor.execute(query_creacion_tabla)
    print("Tabla peliculas creada correctamente (o ya existía).")

except mysql.connector.Error as e:
    # Capturamos y mostramos cualquier error de MySQL
    print("Ocurrió un error al crear la tabla:", e)
